In [ ]:
import GPy
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import os


# MNIST

In [ ]:

# define below the list of pairs of reasoning levels, in the form of reasoning level of attacker vs defender
# use -1 to indicate using R2-B2_Lite and reasoning at level 1
reasoning_level_list = [[0, 0],
                        [1, 0],
                        [-1, 0],
                        [0, 1],
                        [0, -1],
                        [2, 1],
                        [1, 2]]

all_regret_avg_player_1_mean_list = []
all_regret_avg_player_1_stderr_list = []


all_successes_list = []

# level_zero_policy in {"random", "gp_mw"}
level_zero_policy = "random"

log_dir = "results_mnist_" + level_zero_policy

latent_dim = 2

sampling_approximation = 500

N_iter = 5
for r in range(len(reasoning_level_list)):
    reasoning_level_1 = reasoning_level_list[r][0]
    reasoning_level_2 = reasoning_level_list[r][1]

    func_list = np.arange(0, 10)

    all_regret_avg_player_1 = []
    all_first_success_player_1 = []
    successes_list = []
    for i in func_list:
        if reasoning_level_1 == -1:
            log_file_name = log_dir + "/r2b2_mnist_LD_" + str(latent_dim) + "_levels_" \
                    + str(1) + "_" + \
                    str(reasoning_level_2) + "_approx_samples_" + str(sampling_approximation) + \
                    "_iter_" + str(i) + "_r2b2_lite.p"
        elif reasoning_level_2 == -1:
            log_file_name = log_dir + "/r2b2_mnist_LD_" + str(latent_dim) + "_levels_" \
                    + str(reasoning_level_1) + "_" + \
                    str(1) + "_approx_samples_" + str(sampling_approximation) + \
                    "_iter_" + str(i) + "_r2b2_lite.p"
        else:
            log_file_name = log_dir + "/r2b2_mnist_LD_" + str(latent_dim) + "_levels_" \
                    + str(reasoning_level_1) + "_" + \
                    str(reasoning_level_2) + "_approx_samples_" + str(sampling_approximation) + \
                    "_iter_" + str(i) + ".p"

        res = pickle.load(open(log_file_name, "rb"))

        values_1 = res["all"]["values_1"]
        init_1 = res["all"]["init"]["Y_1"]
        values_1 = list(init_1) + values_1
        values_1_avg = np.cumsum(values_1) / (np.arange(len(values_1)) + 1)
        all_regret_avg_player_1.append(values_1_avg)

        flags_1 = res["all"]["F_1"]
        successes_list.append(np.count_nonzero(flags_1))
    
    all_regret_avg_player_1 = np.array(all_regret_avg_player_1)
    all_regret_avg_player_1_mean = np.mean(all_regret_avg_player_1, axis=0)

    all_regret_avg_player_1_stderr = np.std(all_regret_avg_player_1, axis=0) / np.sqrt(all_regret_avg_player_1.shape[0])

    all_regret_avg_player_1_mean_list.append(all_regret_avg_player_1_mean)
    all_regret_avg_player_1_stderr_list.append(all_regret_avg_player_1_stderr)
    
    all_successes_list.append(successes_list)


In [ ]:

all_successes_list = np.array(all_successes_list)
print("Average # of successful attacks over 150 iterations: ", np.mean(all_successes_list, axis=1))


In [ ]:

N_total = len(all_regret_avg_player_1_mean_list)
ub_player_1_list = []
lb_player_1_list = []
for i in range(N_total):
    ub_player_1_list.append(all_regret_avg_player_1_mean_list[i] + all_regret_avg_player_1_stderr_list[i])
    lb_player_1_list.append(all_regret_avg_player_1_mean_list[i] - all_regret_avg_player_1_stderr_list[i])


In [ ]:

def plot_mean_and_CI_with_marker(time_steps, mean, lb, ub, color_mean=None, color_shading=None, marker=None, marker_size=12):
    plt.fill_between(time_steps, ub, lb,
                     color=color_shading, alpha=.2)
    plt.plot(time_steps, mean, color_mean, marker=marker, markersize=marker_size, markevery=10)


In [ ]:

lw = 2.0
plt.rc('font', size=26)
plt.figure(figsize=(9.2, 6))

color_list = ["tab:blue", "tab:orange", "tab:red", "tab:green", "tab:purple", "tab:gray", "tab:olive"]
marker_list = ["v", "D", "s", "o", "x", "^", "<"]

N_total = len(all_regret_avg_player_1_mean_list)
inspect_list = np.arange(N_total)
for i in inspect_list:
    inds = np.arange(len(all_regret_avg_player_1_mean_list[i]))
    plot_mean_and_CI_with_marker(inds, all_regret_avg_player_1_mean_list[i], lb_player_1_list[i], ub_player_1_list[i], \
                     color_mean=color_list[i], color_shading=color_list[i], marker=marker_list[i], marker_size=12)

# plt.legend(("Lv 0 vs. Lv 0", "Lv 1 vs. Lv 0", "Lv 1 vs. Lv 0 (R2-B2-Lite)", "Lv 0 vs. Lv 1", "Lv 0 vs. Lv 1 (R2-B2-Lite)", \
#            "Lv 2 vs. Lv 1", "Lv 1 vs. Lv 2"), \
#           prop={'size':20}, loc='lower left', framealpha=0.7)

axes = plt.gca()
axes.set_xlim([0, len(all_regret_avg_player_1_mean_list[0])])

plt.ylabel("Attack Score")
plt.xlabel("Iterations")

plt.show()


# CIFAR-10

In [ ]:

# define below the list of pairs of reasoning levels, in the form of reasoning level of attacker vs defender
# use -1 to indicate using R2-B2_Lite and reasoning at level 1
reasoning_level_list = [[0, 0],
                        [1, 0],
                        [-1, 0],
                        [0, 1],
                        [0, -1],
                        [2, 1],
                        [1, 2]]

all_regret_avg_player_1_mean_list = []
all_regret_avg_player_1_stderr_list = []


all_successes_list = []

# level_zero_policy in {"random", "gp_mw"}
level_zero_policy = "random"

log_dir = "results_cifar_" + level_zero_policy

latent_dim = 8

sampling_approximation = 1000

N_iter = 5
for r in range(len(reasoning_level_list)):
    reasoning_level_1 = reasoning_level_list[r][0]
    reasoning_level_2 = reasoning_level_list[r][1]

    func_list = np.arange(0, 10)

    all_regret_avg_player_1 = []
    all_first_success_player_1 = []
    successes_list = []
    for i in func_list:
        if reasoning_level_1 == -1:
            log_file_name = log_dir + "/r2b2_cifar_LD_" + str(latent_dim) + "_levels_" \
                    + str(1) + "_" + \
                    str(reasoning_level_2) + "_approx_samples_" + str(sampling_approximation) + \
                    "_iter_" + str(i) + "_r2b2_lite.p"
        elif reasoning_level_2 == -1:
            log_file_name = log_dir + "/r2b2_cifar_LD_" + str(latent_dim) + "_levels_" \
                    + str(reasoning_level_1) + "_" + \
                    str(1) + "_approx_samples_" + str(sampling_approximation) + \
                    "_iter_" + str(i) + "_r2b2_lite.p"
        else:
            log_file_name = log_dir + "/r2b2_cifar_LD_" + str(latent_dim) + "_levels_" \
                    + str(reasoning_level_1) + "_" + \
                    str(reasoning_level_2) + "_approx_samples_" + str(sampling_approximation) + \
                    "_iter_" + str(i) + ".p"

        res = pickle.load(open(log_file_name, "rb"))
            
        values_1 = res["all"]["values_1"]
        init_1 = res["all"]["init"]["Y_1"]
        values_1 = list(init_1) + values_1
        values_1_avg = np.cumsum(values_1) / (np.arange(len(values_1)) + 1)
        all_regret_avg_player_1.append(values_1_avg)

        flags_1 = res["all"]["F_1"]
        successes_list.append(np.count_nonzero(flags_1))
    
    all_regret_avg_player_1 = np.array(all_regret_avg_player_1)
    all_regret_avg_player_1_mean = np.mean(all_regret_avg_player_1, axis=0)

    all_regret_avg_player_1_stderr = np.std(all_regret_avg_player_1, axis=0) / np.sqrt(all_regret_avg_player_1.shape[0])

    all_regret_avg_player_1_mean_list.append(all_regret_avg_player_1_mean)
    all_regret_avg_player_1_stderr_list.append(all_regret_avg_player_1_stderr)
    
    all_successes_list.append(successes_list)


In [ ]:

all_successes_list = np.array(all_successes_list)
print("Average # of successful attacks over 150 iterations: ", np.mean(all_successes_list, axis=1))


In [ ]:

N_total = len(all_regret_avg_player_1_mean_list)
ub_player_1_list = []
lb_player_1_list = []
for i in range(N_total):
    ub_player_1_list.append(all_regret_avg_player_1_mean_list[i] + all_regret_avg_player_1_stderr_list[i])
    lb_player_1_list.append(all_regret_avg_player_1_mean_list[i] - all_regret_avg_player_1_stderr_list[i])


In [ ]:

def plot_mean_and_CI_with_marker(time_steps, mean, lb, ub, color_mean=None, color_shading=None, marker=None, marker_size=12):
    plt.fill_between(time_steps, ub, lb,
                     color=color_shading, alpha=.2)
    plt.plot(time_steps, mean, color_mean, marker=marker, markersize=marker_size, markevery=10)


In [ ]:

lw = 2.0
plt.rc('font', size=26)
plt.figure(figsize=(9.2, 6))

color_list = ["tab:blue", "tab:orange", "tab:red", "tab:green", "tab:purple", "tab:gray", "tab:olive"]
marker_list = ["v", "D", "s", "o", "x", "^", "<"]

N_total = len(all_regret_avg_player_1_mean_list)
inspect_list = np.arange(N_total)
for i in inspect_list:
    inds = np.arange(len(all_regret_avg_player_1_mean_list[i]))
    plot_mean_and_CI_with_marker(inds, all_regret_avg_player_1_mean_list[i], lb_player_1_list[i], ub_player_1_list[i], \
                     color_mean=color_list[i], color_shading=color_list[i], marker=marker_list[i], marker_size=12)

# plt.legend(("Lv 0 vs. Lv 0", "Lv 1 vs. Lv 0", "Lv 1 vs. Lv 0 (R2-B2-Lite)", "Lv 0 vs. Lv 1", "Lv 0 vs. Lv 1 (R2-B2-Lite)", \
#            "Lv 2 vs. Lv 1", "Lv 1 vs. Lv 2"), \
#           prop={'size':20}, loc='lower left', framealpha=0.7)

axes = plt.gca()
axes.set_xlim([0, len(all_regret_avg_player_1_mean_list[0])])

plt.ylabel("Attack Score")
plt.xlabel("Iterations")

plt.show()
